In [6]:
import pandas as pd
import numpy as np

In [15]:
# Directory in which we have saved the data
data_directory = ""

# Read data
X_df = pd.read_csv(data_directory+"BASC197_X_corr.csv")
X = X_df.to_numpy()

Y_df = pd.read_csv(data_directory+"Y.csv")
Y = Y_df.to_numpy()

### Feature selection

In [5]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE

def anova_feature_selection(X_train, y_train, X_test, k):
    """
    Takes training and test data, calculate ANOVA F-score and select k 
    best features, transforming both datasets
    """

    anova_model = SelectKBest(score_func=f_classif, k=k)

    # Learn which are the best feature according to ANOVA F-score
    anova_model.fit(X_train, y_train)

    # transform train input data
    X_train_anova = anova_model.transform(X_train)

    # Transform test input data
    X_test_anova = anova_model.transform(X_test)

    return X_train_anova, X_test_anova, anova_model


def RFE():
    pass

### Cross-validation


In [3]:
from julearn import run_cross_validation
from julearn import PipelineCreator
from julearn.utils import configure_logging
from julearn.viz import plot_scores
from sklearn.model_selection import StratifiedKFold

In [16]:
# Convert data to one DataFrame

def to_dataframe(X,Y):
    df_features = pd.DataFrame(X)

    # Convert the n x 1 array to a DataFrame
    df_labels = pd.DataFrame(Y)

    # Assign column names to the features DataFrame
    num_features = df_features.shape[1]
    feature_columns = [f'feature{i+1}' for i in range(num_features)]
    df_features.columns = feature_columns

    # Assign a column name to the labels DataFrame
    df_labels.columns = ['label']

    # Concatenate the two DataFrames
    data_df = pd.concat([df_features, df_labels], axis=1)

    return data_df, feature_columns

data_df, feature_columns = to_dataframe(X,Y)

In [18]:
# SVM model
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

creator1 = PipelineCreator(problem_type = "classification")
creator1.add("zscore")
creator1.add("svm", kernel="linear", C=[0.1, 1, 10])

creator2 = PipelineCreator(problem_type="classification")
creator2.add("zscore")
creator2.add("svm", kernel=["rbf", "sigmoid"], coef0=[0.1,1,10],gamma=[0.001, 0.01])


scores, model, inspector = run_cross_validation(
    X=feature_columns,
    y="label",
    data=data_df,
    model=[creator1,creator2],
    cv=cv,
    return_train_score=True,
    return_inspector=True
)

The dataframe has 19504 columns. Checking X for consistency might take a while. To skip this checks, set the config flag `disable_x_check` to `True`.
/home/juancruz/miniconda3/envs/neurodc/lib/python3.11/site-packages/julearn/prepare.py:195: RuntimeWarning: The dataframe has 19504 columns. Checking X for consistency might take a while. To skip this checks, set the config flag `disable_x_check` to `True`.
  warn_with_log(
X has 19503 columns. Checking X_types for consistency might take a while. To skip this checks, set the config flag `disable_xtypes_check` to `True`.
/home/juancruz/miniconda3/envs/neurodc/lib/python3.11/site-packages/julearn/prepare.py:465: RuntimeWarning: X has 19503 columns. Checking X_types for consistency might take a while. To skip this checks, set the config flag `disable_xtypes_check` to `True`.
  warn_with_log(
The following columns are not defined in X_types: ['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6', 'feature7', 'feature8', 'feat

KeyboardInterrupt: 

### Tests

In [10]:
X_train = X[:700,:]
Y_train = Y[:700]
X_test = X[700:,:]

In [18]:
X_train_anova, X_test_anova, anova_model = anova_feature_selection(X_train, Y_train.ravel(), X_test, 15000)

In [19]:
X_train_anova.shape

(700, 15000)